In [ ]:
# importing libraries
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.applications.vgg16 import VGG16

from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras import Model,layers

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="pzQhYsonvDzrZfiTVd75")
project = rf.workspace("nagesh-pwywk").project("skin-disease-spiyb")
dataset = project.version(1).download("folder")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.0/70.0 kB 881.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.9.0.80
    Uninstalling opencv-python-headless-4.9.0.80:
      Successfully uninstalled opencv-python-headless-4.9.0.80
  Attempting uninstall: idna
    Found existing installation: idna 3.6
    Uninstalling idna-3.6:
      Successfully uninstalled idna-3.6
  Attempting uninstall: cycler
    Found existing installation: cycler

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Skin-Disease-1 in folder:: 100%|██████████| 4559/4559 [00:00<00:00, 5534.85it/s]


In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_dir = "/content/Skin-Disease-1/train"
test_dir="/content/Skin-Disease-1/test"

In [ ]:
BATCH_SIZE=32
# from tensorflow.keras.applications.VGG16 import preprocess_input
# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   validation_split=0.2
                                   )

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator( rescale = 1.0/255. )

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size = BATCH_SIZE,
                                                    subset='training',
                                                    class_mode = 'categorical',
                                                    target_size = (224,224))

# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  train_datagen.flow_from_directory( train_dir,
                                                          batch_size  = BATCH_SIZE,
                                                          subset='validation',
                                                          class_mode  = 'categorical',
                                                          target_size = (224,224))

test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=(224,224),
                                                  batch_size=1,
                                                  shuffle=False,
                                                  class_mode=None)


Found 3269 images belonging to 9 classes.
Found 814 images belonging to 9 classes.
Found 454 images belonging to 9 classes.


In [ ]:
vggModel =VGG16(include_top=False,weights='imagenet',input_shape=(224,224,3))

58889256/58889256 [==============================] - 1s 0us/step


In [ ]:
for layers in vggModel.layers:
  layers.trainable=False

In [ ]:
x =Flatten()(vggModel.output)
x=tensorflow.keras.layers.Dense(1024,activation="relu")(x)
x=tensorflow.keras.layers.Dense(9,activation="softmax")(x)
vgg_model=  Model(vggModel.input,x)

In [ ]:
from tensorflow.keras.optimizers import Adam,SGD
vgg_model.compile(loss = 'categorical_crossentropy', optimizer =Adam(lr=0.01), metrics = ['accuracy'])



In [ ]:
BATCH_SIZE=32
n_steps=train_generator.samples//BATCH_SIZE
n_val_steps= validation_generator.samples//BATCH_SIZE
# history = model.fit_generator(
#             train_generator,
#             validation_data = validation_generator,
#             steps_per_epoch = 20,
#             epochs = 100,
#             validation_steps =n_val_steps,
#             verbose = 2)
vgg1_history = vgg_model.fit(train_generator, batch_size=BATCH_SIZE,
                            epochs=25,
                            validation_data=validation_generator,
                            steps_per_epoch=n_steps,
                            validation_steps=n_val_steps)

Epoch 1/25
102/102 [==============================] - 77s 663ms/step - loss: 2.3565 - accuracy: 0.4442 - val_loss: 1.2302 - val_accuracy: 0.5325
Epoch 2/25
102/102 [==============================] - 66s 647ms/step - loss: 1.0369 - accuracy: 0.6395 - val_loss: 1.0491 - val_accuracy: 0.5600
Epoch 3/25
102/102 [==============================] - 66s 647ms/step - loss: 0.8877 - accuracy: 0.6840 - val_loss: 1.0114 - val_accuracy: 0.6112
Epoch 4/25
102/102 [==============================] - 67s 654ms/step - loss: 0.7825 - accuracy: 0.7198 - val_loss: 0.9336 - val_accuracy: 0.6538
Epoch 5/25
102/102 [==============================] - 66s 652ms/step - loss: 0.7025 - accuracy: 0.7501 - val_loss: 0.8362 - val_accuracy: 0.6825
Epoch 6/25
102/102 [==============================] - 67s 653ms/step - loss: 0.6668 - accuracy: 0.7612 - val_loss: 0.9461 - val_accuracy: 0.6762
Epoch 7/25
102/102 [==============================] - 66s 645ms/step - loss: 0.6712 - accuracy: 0.7609 - val_loss: 0.9154 - val_ac

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score

true_classes = test_generator.classes
class_indices = train_generator.class_indices  # Remove parentheses
class_indices = dict((v, k) for k, v in class_indices.items())
vgg_preds = vgg_model.predict(test_generator)
pred_classes = np.argmax(vgg_preds, axis=1)

acc = accuracy_score(true_classes, pred_classes)
print("Accuracy: {:.2f}%".format(acc * 100))





454/454 [==============================] - 6s 11ms/step
Accuracy: 87.67%


In [ ]:
import numpy as np
vgg_pred_classes = np.argmax(vgg_preds,axis=1)


In [ ]:
from sklearn.metrics import accuracy_score,precision_score
accuracy =accuracy_score(true_classes,vgg_pred_classes)
print(accuracy*100)

87.66519823788546


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
vgg_model.save("/content/drive/MyDrive/skinvgg16.h5")

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.


In [ ]:
from keras.models import load_model
model =load_model("/content/drive/MyDrive/skinvgg16.h5")

In [ ]:
densenetModel =DenseNet121(include_top=False,weights='imagenet',input_shape=(224,224,3))
for layers in densenetModel.layers:
  layers.trainable=False
x =Flatten()(densenetModel.output)
x=tensorflow.keras.layers.Dense(1024,activation="relu")(x)
x=tensorflow.keras.layers.Dense(9,activation="softmax")(x)
densenet_model=  Model(densenetModel.input,x)
from tensorflow.keras.optimizers import Adam,SGD

densenet_model.compile(loss = 'categorical_crossentropy', optimizer =SGD(lr=0.01), metrics = ['accuracy'])


In [ ]:
BATCH_SIZE=32
n_steps=train_generator.samples//BATCH_SIZE
n_val_steps= validation_generator.samples//BATCH_SIZE
densenet_history = densenet_model.fit(train_generator, batch_size=BATCH_SIZE,
                            epochs=25,
                            validation_data=validation_generator,
                            steps_per_epoch=n_steps,
                            validation_steps=n_val_steps)

Epoch 1/25
102/102 [==============================] - 77s 699ms/step - loss: 1.4496 - accuracy: 0.5839 - val_loss: 0.8176 - val_accuracy: 0.6938
Epoch 2/25
102/102 [==============================] - 66s 650ms/step - loss: 0.6468 - accuracy: 0.7683 - val_loss: 0.9068 - val_accuracy: 0.6913
Epoch 3/25
102/102 [==============================] - 66s 653ms/step - loss: 0.4712 - accuracy: 0.8224 - val_loss: 0.8674 - val_accuracy: 0.6913
Epoch 4/25
102/102 [==============================] - 65s 640ms/step - loss: 0.4607 - accuracy: 0.8394 - val_loss: 0.8441 - val_accuracy: 0.7088
Epoch 5/25
102/102 [==============================] - 58s 565ms/step - loss: 0.3204 - accuracy: 0.8851 - val_loss: 0.7246 - val_accuracy: 0.7563
Epoch 6/25
102/102 [==============================] - 57s 564ms/step - loss: 0.2917 - accuracy: 0.8953 - val_loss: 0.9053 - val_accuracy: 0.7150
Epoch 7/25
102/102 [==============================] - 58s 565ms/step - loss: 0.2561 - accuracy: 0.9055 - val_loss: 0.8243 - val_ac

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score

true_classes = test_generator.classes
class_indices = train_generator.class_indices  # Remove parentheses
class_indices = dict((v, k) for k, v in class_indices.items())
densenet_preds = densenet_model.predict(test_generator)
pred_classes = np.argmax(densenet_preds, axis=1)

acc = accuracy_score(true_classes, pred_classes)
print("Accuracy: {:.2f}%".format(acc * 100))

454/454 [==============================] - 10s 14ms/step
Accuracy: 92.29%
